In [1]:
import dotenv

dotenv.load_dotenv(override=True);

- store for parent
- vectorstore for children
- embeddings for children

In [2]:
from langchain.storage import InMemoryStore, LocalFileStore
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
vectorstore = Chroma(embedding_function=embeddings, persist_directory='chroma_persist')
# docstore = InMemoryStore()

from langchain.storage._lc_store import create_kv_docstore
fs = LocalFileStore("./store")
docstore = create_kv_docstore(fs)

/tmp/ipykernel_35217/217374399.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
/tmp/ipykernel_35217/217374399.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(embedding_function=embeddings, persist_directory='chroma_persist')


construct retriever

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=docstore,
    child_splitter=RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=40),
    parent_splitter=RecursiveCharacterTextSplitter(chunk_size=2000), 
)

load documents

In [4]:
from typing import List
from langchain_core.documents import Document
from langchain_community.document_loaders import WebBaseLoader

def load_using_webbaseloader(url) -> List[Document]:
    loader = WebBaseLoader(url)

    docs = loader.load()
    
    return docs

docs = load_using_webbaseloader("https://zakon.rada.gov.ua/laws/show/1306-2001-%D0%BF/print")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
retriever.add_documents(docs)

In [58]:
# res = retriever.invoke("Коли можна здійснювати обгін?")
# res = retriever.invoke("З якою швидкістю можна їхати по селу?")

- create chat model, prompt template
- combine chat_model, propt template and retriever

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI

input_context_prompt = ChatPromptTemplate.from_messages([
    ('system', """Ти знавець правил дорожнього руху (ПДР) в Україні. Відповідай на питання беручи до уваги наступний контекст:\n\n{context}"""),
    ('user', """{input}"""),
    ]
)

chat_model = ChatOpenAI()

qa_chain = create_retrieval_chain(retriever, create_stuff_documents_chain(chat_model, input_context_prompt))

In [6]:
res = qa_chain.invoke({'input': "Коли можна здійснювати обгін?"})

In [7]:
res

{'input': 'Коли можна здійснювати обгін?',
 'context': [Document(metadata={'source': 'https://zakon.rada.gov.ua/laws/show/1306-2001-%D0%BF/print', 'title': 'Про Правила дорожнього руху | від 10.10.2001 № 1306 (Текст для друку)', 'description': 'База даних "Законодавство України" містить нормативно-правові документи українського законодавства, термінологію, анотації англійською мовою, переліки первиних актів тощо', 'language': 'uk'}, page_content="10.8. Якщо в місці з'їзду з дороги є смуга гальмування, водій, який має намір повернути на іншу дорогу, повинен своєчасно перестроїтися на цю смугу і знижувати швидкість тільки на ній.\n\nЯкщо в місці в'їзду на дорогу є смуга розгону, водій має рухатися по ній і вливатися в транспортний потік, даючи дорогу транспортним засобам, що рухаються по цій дорозі.\n\n10.9. Під час руху транспортного засобу заднім ходом водій не повинен створювати небезпеки чи перешкод іншим учасникам руху. Для забезпечення безпеки руху він у разі потреби повинен зверну

In [20]:
chat_model.model_name

'gpt-3.5-turbo'

In [18]:
res['context'][0].page_content.__len__()

1894

In [101]:
print(res['answer'])

Обгін можна здійснювати у таких випадках:

- Якщо жоден з водіїв транспортних засобів, які рухаються за вами і яким може бути створено перешкоду, не розпочав обгону;
- Водій транспортного засобу, який рухається попереду по тій самій смузі, не подав сигналу про намір повороту ліворуч;
- Смуга зустрічного руху, на яку ви будете виїжджати, вільна від транспортних засобів на достатній для обгону відстані;
- Після обгону зможете, не створюючи перешкоди транспортному засобу, якого обганяєте, повернутися на займану смугу.

Заборонено здійснювати обгін на перехресті, на залізничних переїздах і ближче ніж за 100 м перед ними, ближче ніж за 50 м перед пішохідним переходом у населеному пункті і 100 м поза населеним пунктом, у кінці підйому, на мостах, естакадах, шляхопроводах, крутих поворотах та інших ділянках доріг з обмеженою оглядовістю чи в умовах недостатньої видимості, у тунелях, на дорогах з двома і більше смугами для руху в одному напрямку, за колоною транспортних засобів, позаду якої ру

In [84]:
res = qa_chain.invoke({'input': "З якою швидкістю можна їхати по селу?"})

In [93]:
res = qa_chain.invoke({'input': "Скільки можна їхати по селу?"})

In [94]:
print(res['answer'])

У населених пунктах, в тому числі і в селах, дозволяється рух транспортних засобів із швидкістю не більше 50 км/год (пункт 12.4 ПДР). Таким чином, у селі можна їхати не швидше, ніж 50 км/год.


No RAG

In [96]:
no_rag_prompt = ChatPromptTemplate.from_messages([
    ('system', """Ти знавець правил дорожнього руху (ПДР) в Україні. Відповідай на питання згідно ПДР"""),
    ('user', """{input}"""),
    ]
)
no_rag_qa = no_rag_prompt | chat_model

In [97]:
no_rag_qa.invoke("Скільки можна їхати по селу?")

AIMessage(content='Згідно ПДР, у населених пунктах (включаючи села) дозволено рухатися зі швидкістю до 60 км/год, якщо інше не передбачено дорожніми знаками.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 83, 'total_tokens': 164, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9c2ddd99-0af0-472f-9299-aec0fa50d1d7-0', usage_metadata={'input_tokens': 83, 'output_tokens': 81, 'total_tokens': 164, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [98]:
no_rag_qa.invoke("Коли можна здійснювати обгін?")

AIMessage(content='Обгін дозволяється здійснювати тільки зліва. Обгін можна здійснити лише в разі, якщо ви переконались, що вільний простір вистачить для безпечного обгону, і водій, якого ви обганяєте, не намагається виконати обгін або поворот наліво. Також важливо дотримуватись швидкісного режиму під час обгону і не порушувати інші правила дорожнього руху.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 202, 'prompt_tokens': 86, 'total_tokens': 288, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9de4b829-6a10-4972-9ab4-107071b23b56-0', usage_metadata={'input_tokens': 86, 'output_tokens': 202, 'total_tokens': 288, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})